In [19]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

import numpy as np

In [53]:
input_dim = 10
num_topics = 20
num_hidden = 512
alpha = 1/20.

In [59]:
mu1 = np.log(alpha) - 1/num_topics * num_topics * np.log(alpha)
sigma1 = 1. / alpha * (1 - 2. / num_topics) + 1 / (num_topics ** 2) * num_topics / alpha
inv_sigma1 = 1. / sigma1
log_det_sigma = num_topics * np.log(sigma1)

In [105]:
h_dim = num_topics
a = np.ones(h_dim).astype(np.float32)
prior_mean = np.log(a) - np.mean(np.log(a))
prior_var = (((1.0 / a) * (1 - (2.0 / h_dim))) + (1.0 / (h_dim * h_dim)) * np.sum((1.0 / a)))
log_prior_var = np.log(prior_var)

In [106]:
x = layers.Input(shape=(input_dim,))
h = layers.Dense(num_hidden, activation="softplus")(x)
h = layers.Dense(num_hidden, activation="softplus")(h)

z_mean = layers.Dense(num_topics)(h)
z_mean = layers.BatchNormalization()(z_mean)

z_log_var = layers.Dense(num_topics)(h)
z_log_var = layers.BatchNormalization()(z_log_var)

def sampling(args):
    z_mean, z_log_var = args
    
    epsilon = K.random_normal((num_topics,), mean=0.0, stddev=1.0)
    
    return z_mean + K.exp(z_log_var / 2) * epsilon

unnormalized_z = layers.Lambda(sampling, output_shape=(num_topics,))([z_mean, z_log_var])

theta = layers.Activation("softmax")(unnormalized_z)
theta = layers.Dropout(0.5)(theta)

doc = layers.Dense(input_dim)(theta)
doc = layers.BatchNormalization()(doc)
doc = layers.Activation("softmax")(doc)

In [107]:
def vae_loss(y_true, y_pred):
    reconstruct = K.sum(y_true * K.log(y_pred + 1e-10), axis=1)
    
    var_division = K.exp(z_log_var) / prior_var
    diff = z_mean - prior_mean
    diff_term = diff * diff / prior_var
    log_var_division = log_prior_var - z_log_var
    
    kl = -0.5 * (K.sum(var_division + diff_term + log_var_division, 1) - h_dim)
    
    return -K.mean(reconstruct + kl)

In [108]:
vae = Model(x, doc)

In [109]:
vae.compile(
    optimizer=tf.train.AdadeltaOptimizer(learning_rate=0.01),
    loss=vae_loss
)

In [110]:
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 512)          5632        input_6[0][0]                    
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 512)          262656      dense_24[0][0]                   
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 20)           10260       dense_25[0][0]                   
__________________________________________________________________________________________________
dense_27 (

In [111]:
N_SAMPLES = 100

In [112]:
X = np.random.rand(N_SAMPLES, input_dim)

In [113]:
vae.fit(X, X, epochs=100)

Epoch 1/100
100/100 [==============================] - 1s 15ms/step - loss: 17.6960
Epoch 2/100
100/100 [==============================] - 0s 237us/step - loss: 17.3327
Epoch 3/100
100/100 [==============================] - 0s 249us/step - loss: 17.3553
Epoch 4/100
100/100 [==============================] - 0s 249us/step - loss: 17.2746
Epoch 5/100
100/100 [==============================] - 0s 262us/step - loss: 17.3649
Epoch 6/100
100/100 [==============================] - 0s 280us/step - loss: 17.1095
Epoch 7/100
100/100 [==============================] - 0s 301us/step - loss: 17.2908
Epoch 8/100
100/100 [==============================] - 0s 302us/step - loss: 17.2712
Epoch 9/100
100/100 [==============================] - 0s 298us/step - loss: 17.1541
Epoch 10/100
100/100 [==============================] - 0s 286us/step - loss: 17.0532
Epoch 11/100
100/100 [==============================] - 0s 301us/step - loss: 16.9844
Epoch 12/100
100/100 [==============================] - 0s 308us

100/100 [==============================] - 0s 387us/step - loss: 13.9914
Epoch 97/100
100/100 [==============================] - 0s 406us/step - loss: 14.0407
Epoch 98/100
100/100 [==============================] - 0s 363us/step - loss: 14.1579
Epoch 99/100
100/100 [==============================] - 0s 200us/step - loss: 13.7246
Epoch 100/100
100/100 [==============================] - 0s 195us/step - loss: 14.0589
